# Analysis for Group Project

### A single dataset is used for all the crop prices 

this dataset displays a time series of prices for different Canadian agricultural products over time.

[Farm product prices, crops and livestock](https://open.canada.ca/data/en/dataset/d5614095-e77a-4cb4-a5e6-9f8bff067c9f)


In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm


# here is a util we'll use to plot graphs of prices
def _plot_time_series_from_df(df, x, y, z, product_name):
    sns.lineplot(data=df, x=x, y=y, hue=z)
    plt.title(f"Time Series of Prices for {product_name}")
    plt.xlabel("Date")
    plt.ylabel(df["UOM"].iloc[0])


In [ ]:
# CSV was downloaded and added to the git repo
# Read in the prices CSV
prices_df = pd.read_csv("./data/32100077.csv")
prices_df.head()

In [ ]:
# DATA PREP

# if you look at the DF, there is a date column in the format of YYYY-MM
## let's turn this into a proper datetime object 
prices_df["REF_DATE"] = pd.to_datetime(prices_df["REF_DATE"].apply(lambda x: dt.datetime.strptime(x, "%Y-%m")))
# let's also add an integer column for Year
## this column will be useful for grouping in our analysis
prices_df["YEAR"] = prices_df["REF_DATE"].apply(lambda x: x.year)
## convert UOM to category data type
prices_df["UOM"] = prices_df["UOM"].astype("category")
## convert SCALAR_FACTOR to category data type
prices_df["SCALAR_FACTOR"] = prices_df["SCALAR_FACTOR"].astype("category")
prices_df.head()


In [ ]:
# let's look at the shape of the data frames
print(prices_df.shape)

In [ ]:
# Just so that we have a comprehensive list of the different products in this dataset, let's print out a unique lit of the `Farm products` column 

products = pd.Series(prices_df["Farm products"].sort_values().unique(), name="products")
products

# Part 3: Wheat


In [ ]:
# Prices

# DATA PREP

# define a `products` variable that will be used to select the rows of interest
product = "Wheat (except durum wheat) [1121111]"

# create a boolean mask that will select chickens for meat and rows where the `VALUE` column is not null 
mask = (prices_df["Farm products"] == product) & (prices_df["VALUE"].notnull())
wheat_prices_df = prices_df[mask]
wheat_prices_df.head()

In [ ]:
# we need to make sure we are only dealing with a single category for UoM and Scale
## sometimes these datasets will have the same data in a subsequent row for a different scale. e.g., `Dollars per kg` vs. `dollars per pound`.
assert wheat_prices_df["Farm products"].unique().__len__() == 1
assert wheat_prices_df["UOM"].unique().__len__() == 1
assert wheat_prices_df["SCALAR_FACTOR"].unique().__len__() == 1

# if there is no exception raised, we are good to go!  

When we look at the historical prices, Newfoundland looks very different from the other provinces. It is clear that NL is distinct from the rest of the country which is comparatively in lock step. 

That being considered, it would make sense to exclude this province from our analysis.

Furthermore, in order to work with the data, it will be easier to have all the price data aggregated into a single response. 

The graphs below show the three states of the data:

1. The raw price data from all the provinces
2. The raw price data from all the provinces except for NL which was excluded
3. An aggregated price curve - the annual mean for all provinces, excluding NL


In [ ]:
# define the fig size
plt.figure(figsize=(20, 8))

# plot with all provinces
plt.subplot(1, 3, 1)
_plot_time_series_from_df(wheat_prices_df, "REF_DATE", "VALUE", "GEO", "Chickens for Meat by Province, All Provinces")

# plot with all provinces, excluding newfoundland
plt.subplot(1, 3, 2)
mask = (wheat_prices_df["GEO"] != "Newfoundland and Labrador")
hog_prices_df = wheat_prices_df[mask]
_plot_time_series_from_df(hog_prices_df, "REF_DATE", "VALUE", "GEO", "Chickens for Meat by Province, Excluding NL")

# plot with all provinces, combined into a single variable
plt.subplot(1, 3, 3)
hog_prices_df.groupby("YEAR")["VALUE"].mean()

# convert to time series - this will be used later on for the final analysis
ser_hog_prices = pd.Series(
    data=hog_prices_df.groupby("YEAR")["VALUE"].mean(numeric_only=True).values,
    index=hog_prices_df.groupby("YEAR").mean(numeric_only=True).index.to_series().apply(lambda x: dt.datetime.strptime(str(x), "%Y")),
    name="prices"
)
# Jeush, please take note of the transformation needed to create the index on the time series

# labels for plot
labels = {
    "title": f"Aggregated Price of Hogs in Canada ({ser_hog_prices.index.min().year}-{ser_hog_prices.index.max().year})",
    "xlabel": "Date",
    "ylabel": "Dollars per hundredweight",
}
ser_hog_prices.plot(**labels)



Now we need some production data. Let's use this dataset: 
[Hogs, sheep and lambs, farm and meat production](https://open.canada.ca/data/en/dataset/28a34385-92a2-4934-8899-2cc85fcc4786)

In [ ]:

# from https://open.canada.ca/data/en/dataset/28a34385-92a2-4934-8899-2cc85fcc4786

# CSV was downloaded and added to the git repo
# This will be the response variable
production_df = pd.read_csv("./data/32100126.csv")

# let's take a look
print(production_df.shape)
production_df.head()


In [ ]:
# DATA PREP

# convert the year to a datetime obj
production_df["REF_DATE"] = pd.to_datetime(production_df["REF_DATE"].apply(lambda x: dt.datetime.strptime(str(x), "%Y")))

# again let's create a boolean mask to select the rows of interest
mask = (
        (production_df["Estimates"] == "Estimated farm output") &  # let's only look at the estimated form output
        (production_df["Livestock"] == "Hogs") & # only the hogs
        (production_df["VALUE"].notnull())  # only rows that have production values
)
production_df = production_df[mask]
print(production_df.shape)
production_df.head()


In [ ]:
# looks like we have 103 observations...

# Now let's graph the time series
ser_hog_production = pd.Series(data=production_df["VALUE"].values, index=production_df["REF_DATE"], name="production")

# the scale of the data is in thousands. Let's convert to millions
ser_hog_production = ser_hog_production * 1e3 / 1e6

plt.figure(figsize=(5, 5))
# chicken count
plt.subplot(1, 1, 1)
labels = dict(
    xlabel="Date",
    ylabel="Eastimated farm output, in millions",
    title=f"Estimated Output of Hogs in Canada ({ser_hog_production.index.min().year}-{ser_hog_production.index.max().year})"
)
ser_hog_production.plot(**labels)


Now that we have a time series for prices and production, let's merge them into a single dataframe.


In [ ]:
final_df = pd.DataFrame(data={"prices": ser_hog_prices}).join(ser_hog_production)
final_df['Eins'] = np.ones((len(final_df),))
final_df = final_df.dropna()
final_df


### unlike the previous dataset, there was more overlap between the two time series

We have 38 rows of data that can be used in the analysis


In [ ]:

# let's use a linear regression to explore the relationship between prices and production

# a scatter plot and the OLS regression line plotted
sns.lmplot(data=final_df, x="prices", y="production")
plt.title("Effect of Prices on Estimated Hog Output in Canada")
plt.ylabel("Estimated farm output, in millions")
plt.xlabel("Dollars per hundredweight of meat")

In [ ]:
# finally let's look at the results from the regression

Y = final_df["production"]
X = final_df[['prices', 'Eins']]
results = sm.OLS(Y, X).fit()
results.summary()



### linear regression results:

The F-staticic for this model was `0.3441` and the chances of observing this statistic under a normal distribution is approximately 56%. Therefore, with a p-value set to 0.05, we are not in a position to reject the null hypothesis. 



In [ ]:

import math

final_df = new_df.join(ser_chicken_per_farms).dropna()
final_df["log_prices"] = final_df["prices"].apply(lambda x: math.log(x))

In [ ]:

sns.lmplot(data=final_df, x="prices", y="counts")
plt.title("Effect of prices on number of chickens per farm")


In [ ]:
sns.lmplot(data=final_df, x="prices", y="counts")


# OK, let's do everything over again but with swine 

In [ ]:
mask = products.str.lower().str.contains("swine") | products.str.lower().str.contains("hog") | products.str.lower().str.contains("pig")
print(products[mask])
# let's start with just meat
mask = (prices_df["Farm products"] == "Hogs [111121]") & (prices_df["VALUE"].notnull())
hog_prices_df = prices_df[mask]
# make sure we are only dealing with a single category for UoM and Scale
print(hog_prices_df["UOM"].unique())
print(hog_prices_df["SCALAR_FACTOR"].unique())
# view the DF
hog_prices_df



In [ ]:

_plot_time_series_from_df(hog_prices_df, "REF_DATE", "VALUE", "GEO", "Hog Prices by Province")


In [ ]:
### again, NL is an outlier.. we should remove them
mask = (hog_prices_df["GEO"] != "Newfoundland and Labrador")
hog_prices_df = hog_prices_df[mask]
_plot_time_series_from_df(hog_prices_df, "REF_DATE", "VALUE", "GEO", "Hog Prices by Province, excluding NL")


In [ ]:
# aggregate over all of canada

hog_prices_df.groupby("YEAR")["VALUE"].mean()
# convert to time series
ser_hog_prices = pd.Series(
    data=hog_prices_df.groupby("YEAR")["VALUE"].mean(numeric_only=True).values,
    index=hog_prices_df.groupby("YEAR").mean(numeric_only=True).index.to_series().apply(lambda x: dt.datetime.strptime(str(x), "%Y")),
    name="prices"
)
# the above is in CAD/hundredweight and should be converted to CAD/kilogram: 1 hundredweight = 45.36 kg
ser_hog_prices = ser_hog_prices / 45.36
ser_hog_prices

In [ ]:
labels["ylabel"] = "Dollars per kilogram"
labels["title"] = f"Price of Hogs in Canada ({ser_hog_prices.index.min().year}-{ser_hog_prices.index.max().year})"
ser_hog_prices.plot(**labels)

In [ ]:
# now we have to find a swine production dataset
# from https://open.canada.ca/data/en/dataset/28a34385-92a2-4934-8899-2cc85fcc4786

# This will be the response variable
df_raw = pd.read_csv("./data/32100126.csv")
# convert the year to a datetime obj
df_raw["REF_DATE"] = pd.to_datetime(df_raw["REF_DATE"].apply(lambda x: dt.datetime.strptime(str(x), "%Y")))

mask = ((df_raw["Estimates"] == "Estimated farm output") & (df_raw["Livestock"] == "Hogs"))
df_hogs_count = df_raw[mask]
ser_hogs_count = pd.Series(data=df_hogs_count["VALUE"].values, index=df_hogs_count["REF_DATE"], name="counts")
ser_hogs_count = ser_hogs_count * 1e3 / 1e6
ser_hogs_count


In [ ]:

labels = dict(
    xlabel="Date",
    ylabel="Millions of heads",
    title=f"Number of Hogs Output in Canada ({ser_hogs_count.index.min().year}-{ser_hogs_count.index.max().year})"
)
ser_hogs_count.plot(**labels)

In [ ]:
final_df = pd.DataFrame(data={"prices": ser_hog_prices}).join(ser_hogs_count).dropna()
final_df['Eins'] = np.ones((len(final_df),))

final_df
sns.lmplot(data=final_df, x="prices", y="counts")

In [ ]:
final_df = pd.DataFrame(data={"prices": ser_hog_prices}).join(ser_hogs_count).dropna()
final_df['Eins'] = np.ones((len(final_df),))

final_df

In [ ]:
Y = final_df["counts"]
X = final_df[['prices', 'Eins']]
results = sm.OLS(Y, X).fit()
print(results.summary())


# finally let's look at a grain... These crops might be more responsive to fluctuations in price?

In [ ]:
mask = products.str.lower().str.contains("wheat")
print(products[mask])
# let's start with just meat
mask = (prices_df["Farm products"] == "Wheat (except durum wheat) [1121111]") & (prices_df["VALUE"].notnull())
grain_prices_df = prices_df[mask]
grain_prices_df

In [ ]:
_plot_time_series_from_df(grain_prices_df, "REF_DATE", "VALUE", "GEO", "Wheat Prices by Province")


In [ ]:

grain_prices_df.groupby("YEAR")["VALUE"].mean()
# convert to time series
ser_grain_prices = pd.Series(
    data=grain_prices_df.groupby("YEAR")["VALUE"].mean(numeric_only=True).values,
    index=grain_prices_df.groupby("YEAR").mean(numeric_only=True).index.to_series().apply(lambda x: dt.datetime.strptime(str(x), "%Y")),
    name="prices"
)
# the above is in CAD/hundredweight and should be converted to CAD/kilogram: 1 hundredweight = 45.36 kg
ser_grain_prices

In [ ]:
labels["ylabel"] = "Dollars per metric ton"
labels["title"] = f"Price of Wheat in Canada ({ser_grain_prices.index.min().year}-{ser_grain_prices.index.max().year})"
ser_grain_prices.plot(**labels)

In [ ]:
# now we have to find a oat production dataset
# from https://open.canada.ca/data/en/dataset/dd8ffdfb-d9fb-4f16-be28-50632a64d95c

# This will be the response variable
df_raw = pd.read_csv("./data/32100015.csv")
# convert the year to a datetime obj
df_raw["REF_DATE"] = pd.to_datetime(df_raw["REF_DATE"].apply(lambda x: dt.datetime.strptime(str(x), "%Y-%m")))
df_raw["YEAR"] = df_raw["REF_DATE"].apply(lambda x: x.year)
df_raw["MONTH"] = df_raw["REF_DATE"].apply(lambda x: x.month)

# df_raw

mask = df_raw["Type of crop"].str.lower().str.contains("wheat")
print(df_raw["Type of crop"][mask].unique())

mask = ((df_raw["Type of crop"] == "Wheat, excluding durum") & (df_raw["Farm supply and disposition of grains"] == "Production") & (
        df_raw["GEO"] == "Canada") & (df_raw["MONTH"] == 12))
df_grain_count = df_raw[mask]

# df_grain_count_grouped = df_grain_count.groupby("YEAR").first()
# df_grain_count_grouped
ser_grain_prod = pd.Series(data=df_grain_count["VALUE"].values, index=df_grain_count["REF_DATE"], name="production")
# # ser_hogs_count = ser_hogs_count*1e3/1e6
ser_grain_prod


In [ ]:

labels = dict(
    xlabel="Date",
    ylabel="Thousands of metric tonnes",
    title=f"Production of Wheat in Canada ({ser_grain_prod.index.min().year}-{ser_grain_prod.index.max().year})"
)
ser_grain_prod.plot(**labels)

In [ ]:
final_df = pd.DataFrame(data={"prices": ser_grain_prices}).join(ser_grain_prod).dropna()
final_df

In [ ]:
ser_grain_prod = ser_grain_prod.resample("YS").first()
# ser_grain_prod = ser_grain_prod.shift(-1)

In [ ]:
final_df = pd.DataFrame(data={"prices": ser_grain_prices}).join(ser_grain_prod).dropna()
final_df['Eins'] = np.ones((len(final_df),))
final_df

In [ ]:
sns.lmplot(data=final_df, x="prices", y="production")

In [ ]:


Y = final_df["production"]
X = final_df[['prices', 'Eins']]
results = sm.OLS(Y, X).fit()
print(results.summary())

